In [ ]:
!pip install pingouin

import pandas as pd
import pingouin as pg

def calculate_icc(csv_file):
    data = pd.read_csv(csv_file)
    data_long = data.melt(var_name='raters', value_name='ratings', ignore_index=False)
    data_long.reset_index(inplace=True)
    data_long.rename(columns={'index': 'targets'}, inplace=True)
    icc_results = pg.intraclass_corr(data=data_long, targets='targets', raters='raters', ratings='ratings')
    return icc_results

def main_icc(csv_file):
    icc_results = calculate_icc(csv_file)
    print(icc_results.set_index('Type'))

# 使用例
main_icc('deta.csv')
